### 1 - Import libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
import copy

# For reproducibility
np.random.seed(42)
torch.manual_seed(42)

### 2 - Prepare Your Data

In [2]:
# Assuming your data preparation steps are done here
num_simulations = 100
train_ratio = 0.8

# Load npy data from training_data folder
pressure_data = np.load('training_data/pressure_data.npy')
velocity_data = np.load('training_data/velocity_data.npy')

pressure_flattened = pressure_data.reshape(num_simulations, -1)
velocity_flattened = velocity_data.reshape(num_simulations, -1)

data_flattened = np.hstack((pressure_flattened, velocity_flattened))

# Convert to PyTorch tensors
data_tensor = torch.tensor(data_flattened, dtype=torch.float32)

# Split the data
train_size = int(train_ratio * num_simulations)
test_size = num_simulations - train_size
train_dataset, test_dataset = random_split(data_tensor, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### 3 - Define the Autoencoder Model

In [4]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.ReLU(),
            nn.Linear(1024, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

# Define model, loss, and optimizer
input_dim = data_flattened.shape[1]  # 15000
latent_dim = 256
autoencoder = Autoencoder(input_dim, latent_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)


### 4 - Train the Autoencoder

In [5]:
# Training the Autoencoder
num_epochs = 10
best_model_wts = copy.deepcopy(autoencoder.state_dict())
best_loss = float('inf')

for epoch in range(num_epochs):
    autoencoder.train()
    train_loss = 0.0

    for data in train_loader:
        inputs = data
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)

    autoencoder.eval()
    val_loss = 0.0

    with torch.no_grad():
        for data in test_loader:
            inputs = data
            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            val_loss += loss.item() * inputs.size(0)

    val_loss /= len(test_loader.dataset)

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Save the best model
    if val_loss < best_loss:
        best_loss = val_loss
        best_model_wts = copy.deepcopy(autoencoder.state_dict())

# Load the best model weights
autoencoder.load_state_dict(best_model_wts)

# Extract the encoder
encoder = autoencoder.encoder

# Save the trained autoencoder
torch.save(autoencoder.state_dict(), 'model_saved/autoencoder_10epoches.pth')

Epoch 1/10, Train Loss: 0.2073, Val Loss: 0.0945
Epoch 2/10, Train Loss: 0.0609, Val Loss: 0.0413
Epoch 3/10, Train Loss: 0.0446, Val Loss: 0.0514
Epoch 4/10, Train Loss: 0.0510, Val Loss: 0.0487
Epoch 5/10, Train Loss: 0.0473, Val Loss: 0.0429
Epoch 6/10, Train Loss: 0.0410, Val Loss: 0.0336
Epoch 7/10, Train Loss: 0.0292, Val Loss: 0.0240
Epoch 8/10, Train Loss: 0.0223, Val Loss: 0.0204
Epoch 9/10, Train Loss: 0.0191, Val Loss: 0.0177
Epoch 10/10, Train Loss: 0.0172, Val Loss: 0.0158


In [6]:
num_epochs = 20

for epoch in range(num_epochs):
    autoencoder.train()
    train_loss = 0.0

    for data in train_loader:
        inputs = data
        optimizer.zero_grad()
        outputs = autoencoder(inputs)
        loss = criterion(outputs, inputs)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)

    train_loss /= len(train_loader.dataset)

    autoencoder.eval()
    val_loss = 0.0

    with torch.no_grad():
        for data in test_loader:
            inputs = data
            outputs = autoencoder(inputs)
            loss = criterion(outputs, inputs)
            val_loss += loss.item() * inputs.size(0)

    val_loss /= len(test_loader.dataset)

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Save the best model
    if val_loss < best_loss:
        best_loss = val_loss
        best_model_wts = copy.deepcopy(autoencoder.state_dict())

# Load the best model weights
autoencoder.load_state_dict(best_model_wts)

# Extract the encoder
encoder = autoencoder.encoder

# Save the trained autoencoder
torch.save(autoencoder.state_dict(), 'model_saved/autoencoder_30epoches.pth')

Epoch 1/20, Train Loss: 0.0158, Val Loss: 0.0156
Epoch 2/20, Train Loss: 0.0155, Val Loss: 0.0155
Epoch 3/20, Train Loss: 0.0152, Val Loss: 0.0150
Epoch 4/20, Train Loss: 0.0149, Val Loss: 0.0148
Epoch 5/20, Train Loss: 0.0148, Val Loss: 0.0147
Epoch 6/20, Train Loss: 0.0146, Val Loss: 0.0146
Epoch 7/20, Train Loss: 0.0145, Val Loss: 0.0146
Epoch 8/20, Train Loss: 0.0145, Val Loss: 0.0145
Epoch 9/20, Train Loss: 0.0144, Val Loss: 0.0145
Epoch 10/20, Train Loss: 0.0144, Val Loss: 0.0144
Epoch 11/20, Train Loss: 0.0144, Val Loss: 0.0144
Epoch 12/20, Train Loss: 0.0144, Val Loss: 0.0144
Epoch 13/20, Train Loss: 0.0144, Val Loss: 0.0144
Epoch 14/20, Train Loss: 0.0144, Val Loss: 0.0144
Epoch 15/20, Train Loss: 0.0144, Val Loss: 0.0144
Epoch 16/20, Train Loss: 0.0143, Val Loss: 0.0144
Epoch 17/20, Train Loss: 0.0143, Val Loss: 0.0144
Epoch 18/20, Train Loss: 0.0143, Val Loss: 0.0144
Epoch 19/20, Train Loss: 0.0143, Val Loss: 0.0144
Epoch 20/20, Train Loss: 0.0143, Val Loss: 0.0144


### 5. Train the Predictor Model

In [8]:
# Load design parameters from training_data folder
design_parameters = np.load('training_data/displacement_data.npy')
design_parameters_tensor = torch.tensor(design_parameters, dtype=torch.float32)

# Split the design parameters
design_train, design_test = random_split(design_parameters_tensor, [train_size, test_size])

# Function to extract tensors from subsets
def get_subset_tensors(subset):
    return subset.dataset[subset.indices]

latent_train = encoder(get_subset_tensors(train_dataset))
latent_test = encoder(get_subset_tensors(test_dataset))

train_design_loader = DataLoader(TensorDataset(design_train, latent_train), batch_size=32, shuffle=True)
test_design_loader = DataLoader(TensorDataset(design_test, latent_test), batch_size=32, shuffle=False)

class Predictor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Predictor, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, output_dim)
        )

    def forward(self, x):
        return self.model(x)

predictor = Predictor(design_parameters.shape[1], latent_dim)
predictor_criterion = nn.MSELoss()
predictor_optimizer = optim.Adam(predictor.parameters(), lr=0.001)

# Training the Predictor
best_predictor_wts = copy.deepcopy(predictor.state_dict())
best_predictor_loss = float('inf')

for epoch in range(num_epochs):
    predictor.train()
    train_loss = 0.0

    for design_data, latent_data in train_design_loader:
        optimizer.zero_grad()
        outputs = predictor(design_data)
        loss = criterion(outputs, latent_data)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * design_data.size(0)

    train_loss /= len(train_design_loader.dataset)

    predictor.eval()
    val_loss = 0.0

    with torch.no_grad():
        for design_data, latent_data in test_design_loader:
            outputs = predictor(design_data)
            loss = criterion(outputs, latent_data)
            val_loss += loss.item() * design_data.size(0)

    val_loss /= len(test_design_loader.dataset)

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')

    # Save the best model
    if val_loss < best_predictor_loss:
        best_predictor_loss = val_loss
        best_predictor_wts = copy.deepcopy(predictor.state_dict())

# Load the best model weights
predictor.load_state_dict(best_predictor_wts)


AttributeError: 'Subset' object has no attribute 'size'

### 6. Predict Flow Fields for New Design Parameters

In [ ]:
# Predict flow fields for new design parameters
new_design_parameters = np.random.rand(1, design_parameters.shape[1])
new_design_parameters_tensor = torch.tensor(new_design_parameters, dtype=torch.float32)

# Predict latent space representation
latent_pred_new = predictor(new_design_parameters_tensor)

# Decode the latent representation to obtain the flow fields
predicted_flow_fields = autoencoder.decoder(latent_pred_new)
predicted_flow_fields = predicted_flow_fields.detach().numpy().reshape(5000, 3)

print(predicted_flow_fields)